In [1]:
import pandas as pd
import os

os.chdir(r'C:\Users\jf79\Cloned Repos\REAP-Scripts')
import myfuncs.myfuncs as mf


cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry'
os.chdir(cwd)
mf.read_directory()

master_data = pd.read_csv('Netcall Interactions Master Data.csv',low_memory=False)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry
Files in: ['aggregated_interactions.csv', 'aggregated_interactions.xlsx', 'Footfall Data - Strategic Assessment', 'hf_footfall_data.csv', 'london_footfall_data.csv', 'Netcall Interactions Master Data.csv', 'Netcall Interactions Master Data.zip', 'typical_borough_averages.csv', 'typical_ward_averages.csv', 'weekday_lon (annual).csv', 'weekend_lon (annual).csv', 'x.csv']



In [61]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

master_df = master_data.copy()

columns_to_drop = [
    'customername','assistedby',
    'casereference','ipaddress'
]
master_df.drop(columns=columns_to_drop, inplace=True)

master_df['nullcount'] = master_df.isna().sum(axis=1)
master_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(drop.index, inplace=True)
master_df.drop(columns='nullcount', inplace=True)

columns_to_change = ['timeended', 'timequeued']
for column in columns_to_change:
    master_df[column] = pd.to_datetime(master_df[column])

master_df['queuetime'] = master_df['timeended'] - master_df['timequeued']

columns_to_change = [
    'duration','talktime','holdtime',
    'wrap-upduration','agenthandlingtime'
]
master_df[columns_to_change] = master_df[columns_to_change].apply(lambda x: x.str.replace(r'(?<!^)-', '', regex=True))
for column in columns_to_change:
    master_df[column] = pd.to_timedelta(master_df[column])


master_df['dateofqueue'] = master_df['timequeued'].dt.date
master_df.loc[(master_df['type'] == 'Call') & (master_df['queuetime'] < pd.Timedelta(seconds=40)),'isunder40s?'] = 1
master_df['isunder40s?'].fillna(0, inplace=True)
master_df

#########################################################################################################################################################################################
DataFrame: None


C:\Users\jf79\AppData\Local\Temp\ipykernel_18692\1237956904.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  master_df['isunder40s?'].fillna(0, inplace=True)


,interactionid,groupname,type,direction,timequeued,timeended,queue,skill,agent,customercontact,queuetime,duration,talktime,holdtime,wrap-upduration,agenthandlingtime,result,activitycode,transferredto,subject,conversationstarttime,conversationendtime,conversationduration,agentusername,agentid,forwarded,source,conversationid,Year,Month,dateofqueue,isunder40s?
0,2640671,Repair CSC,Email,Inbound,2022-01-01 02:15:00,2022-01-04 16:01:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,Gbenga.Osunkoya@lbhf.gov.uk,3 days 13:46:00,0 days 00:04:05,0 days 00:04:05,0 days 00:00:00,0 days 00:00:04,0 days 00:04:09,Replied,Repair Status,NaN,RE: Inspection item Requires Attention - (BLK) 1-180 Poynter House,01/01/2022 02:15,04/01/2022 16:01,85:46:28,4861,4861.0,0.0,Email,2516337,2022,1,2022-01-01,0.0
1449192,2640673,Customer Services,Email,Inbound,2022-01-01 03:05:38,2022-01-04 09:09:58,Electoral Services,Electoral Services Email,Gillian Hoyle,mccatigan@gmail.com,3 days 06:04:20,0 days 00:00:23,0 days 00:00:22,NaT,NaT,0 days 00:00:22,Other,NaN,NaN,Register to Vote,01/01/2022 03:05:38,04/01/2022 09:09:58,78:04:20,NaN,NaN,1.0,NaN,2516339,2022,1,2022-01-01,0.0
1,2640679,Repair CSC,Email,Inbound,2022-01-01 06:14:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Georgia St Rose,Baboucarr.Gaye@lbhf.gov.uk,3 days 09:51:00,0 days 00:03:35,0 days 00:03:34,0 days 00:00:01,0 days 00:00:09,0 days 00:03:43,Replied,Repair Status,NaN,RE: Norland House Electrical Failure (Job Ref. 1126163/1 emergency within 4hrs),01/01/2022 06:14,04/01/2022 16:05,81:51:20,4861,4861.0,0.0,Email,2516345,2022,1,2022-01-01,0.0
2,2640681,Repair CSC,Email,Inbound,2022-01-01 09:08:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Emma Hicks,orbisreports@orbisprotect.com,3 days 06:57:00,0 days 00:03:06,0 days 00:03:06,0 days 00:00:00,0 days 00:00:16,0 days 00:03:22,Transferred,OOH - Related,Mehul Patel,[Orbis Protect] - Unallocated Cases,01/01/2022 09:08,04/01/2022 16:17,79:09:21,971,971.0,0.0,Email,2516347,2022,1,2022-01-01,0.0
3,2640683,Repair CSC,Email,Inbound,2022-01-01 09:38:00,2022-01-04 16:19:00,Repairs customer e-mail,E-Mail Responses - Customer,Emma Hicks,noreply@lbhf.gov.uk,3 days 06:41:00,0 days 00:14:11,0 days 00:14:10,0 days 00:00:01,0 days 00:01:00,0 days 00:15:10,Replied,Repair Status,NaN,Form submission from: Housing repair notification form,01/01/2022 09:38,04/01/2022 16:19,78:41:00,971,971.0,0.0,Email,2516349,2022,1,2022-01-01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2190972,7733773,Customer Services,Call,Inbound,2025-02-24 17:00:17,2025-02-24 17:04:15,Parking Permits Other,Parking Permits Other,NaN,07502345656,0 days 00:03:58,0 days 00:00:00,0 days 00:00:00,NaT,NaT,0 days 00:00:00,Abandoned,NaN,NaN,NaN,24/02/2025 17:00:17,24/02/2025 17:04:15,00:03:58,NaN,NaN,NaN,NaN,7364921,2025,2,2025-02-24,0.0
2190973,7733779,Customer Services,Call,Inbound,2025-02-24 17:02:02,2025-02-24 17:08:13,Complaints Council Services,Complaints Council Services,Sandra Sappleton,07932408161,0 days 00:06:11,0 days 00:05:57,0 days 00:05:57,NaT,NaT,0 days 00:10:57,Answered,New complaint,NaN,NaN,24/02/2025 17:02:02,24/02/2025 17:08:13,00:06:11,NaN,NaN,NaN,NaN,7364927,2025,2,2025-02-24,0.0
2190974,7733783,Customer Services,Call,Outbound,2025-02-24 17:02:57,2025-02-24 17:03:24,CTAX Change of Circumstance Move In Out,CTAX Change of Circumstance Move In Out,Lorna Bernard,07533146505,0 days 00:00:27,0 days 00:00:21,0 days 00:00:21,NaT,NaT,0 days 00:00:34,Failed,Resolved at first point of contact,NaN,NaN,24/02/2025 17:02:57,24/02/2025 17:03:24,00:00:27,NaN,NaN,NaN,NaN,7364931,2025,2,2025-02-24,1.0
2190975,7733837,Customer Services,Call,Inbound,2025-02-24 17:35:32,2025-02-24 17:37:22,Cost of Living Crisis,Cost of Living Crisis,Faraaz Sharif,07745861167,0 days 00:01:50,0 days 00:01:36,0 days 00:01:36,NaT,NaT,0 days 00:02:06,Answered,Sign Pos

Unique values per field:


,Field Name,No. of Unique Values
0,interactionid,2352644
1,groupname,47
2,type,3
3,direction,2
4,timequeued,1918478
5,timeended,1925179
6,queue,186
7,skill,198
8,agent,759
9,customercontact,331099



Number of duplicate rows:
0 

<class 'pandas.core.frame.DataFrame'>
Index: 2352644 entries, 0 to 2190976
Data columns (total 32 columns):
 #   Column                 Non-Null Count    Dtype          
---  ------                 --------------    -----          
 0   interactionid          2352644 non-null  int64          
 1   groupname              2352644 non-null  object         
 2   type                   2352644 non-null  object         
 3   direction              2352644 non-null  object         
 4   timequeued             2352644 non-null  datetime64[ns] 
 5   timeended              2352644 non-null  datetime64[ns] 
 6   queue                  2352644 non-null  object         
 7   skill                  2149588 non-null  object         
 8   agent                  1881514 non-null  object         
 9   customercontact        2346422 non-null  object         
 10  queuetime              2352644 non-null  timedelta64[ns]
 11  duration               2352644 non-null  timedelta

None


Summary statistics:


,interactionid,timequeued,timeended,queuetime,duration,talktime,holdtime,wrap-upduration,agenthandlingtime,forwarded,conversationid,Year,Month,isunder40s?
count,2.352644e+06,2352644,2352644,2352644,2352644,2352644,1449192,1449192,2352644,328308.000000,2.352644e+06,2.352644e+06,2.352644e+06,2.352644e+06
mean,5.244306e+06,2023-07-14 13:01:37.330373632,2023-07-14 20:59:16.693251584,0 days 07:57:39.362878106,0 days 00:14:49.042576352,0 days 00:04:10.091868127,0 days 00:17:02.661385792,0 days 00:01:23.579939028,0 days 00:06:03.084856017,0.051513,4.980235e+06,2.023057e+03,6.231758e+00,1.093803e-01
min,2.640671e+06,2022-01-01 02:15:00,2022-01-04 07:40:00,-1 days +23:59:50,0 days 00:00:00,-2 days +18:03:37,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0.000000,2.516235e+06,2.022000e+03,1.000000e+00,0.000000e+00
25%,3.986000e+06,2022-09-27 10:08:00,2022-09-27 12:16:17.500000,0 days 00:02:00,0 days 00:00:19,0 days 00:00:15,0 days 00:00:00,0 days 00:00:02,0 days 00:00:40,0.000000,3.803716e+06,2.022000e+03,3.000000e+00,0.000000e+00
50%,5.310662e+06,2023-07-15 17:02:45,2023-07-16 13:47:54.500000,0 days 00:06:20,0 days 00:02:33,0 days 00:02:18,0 days 00:00:00,0 days 00:00:27,0 days 00:03:21,0.000000,5.020911e+06,2.023000e+03,6.000000e+00,0.000000e+00
75%,6.492464e+06,2024-04-17 10:47:39.249999872,2024-04-17 12:37:36,0 days 00:17:56,0 days 00:06:40,0 days 00:05:24,0 days 00:00:01,0 days 00:01:00,0 days 00:07:45,0.000000,6.153226e+06,2.024000e+03,9.000000e+00,0.000000e+00
max,7.733871e+06,2025-02-24 17:56:41,2025-02-24 18:02:15,220 days 03:09:28,41 days 23:37:12,3 days 14:20:23,41 days 23:37:04,3 days 16:24:39,3 days 16:25:12,7.000000,7.365019e+06,2.025000e+03,1.200000e+01,1.000000e+00
std,1.449486e+06,NaN,NaN,2 days 23:38:24.253229017,0 days 05:08:42.680067251,0 days 00:09:40.611551054,0 days 06:31:26.599559422,0 days 00:20:17.222122069,0 days 00:18:58.848130793,0.229956,1.370305e+06,8.886350e-01,3.457116e+00,3.121159e-01


End of data validation
#########################################################################################################################################################################################



In [73]:
aggregated_interactions = master_df.groupby(['direction','type','dateofqueue','groupname','queue','result']).agg(
    number_of_interactions = ('interactionid', 'count'),
    wrap_upduration = ('wrap-upduration','mean'),
    queue_time = ('queuetime','mean'),
    duration = ('queuetime','mean'),
    talk_time = ('talktime','mean'),
    hold_time = ('holdtime','mean'),
    agent_handling_time = ('agenthandlingtime','mean'),
    isunder40s = ('isunder40s?','sum')
).reset_index()

# Function to format timedelta
def format_timedelta(td):
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f'{days:02}.{hours:02}:{minutes:02}:{seconds:02}'
 
columns_to_change = [
    'wrap_upduration','queue_time','duration',
    'talk_time','hold_time','agent_handling_time'
]
for column in columns_to_change:
    aggregated_interactions[column].fillna(pd.Timedelta(0), inplace=True)
    aggregated_interactions[column] = aggregated_interactions[column].apply(format_timedelta)

mf.validate_data(aggregated_interactions)

C:\Users\jf79\AppData\Local\Temp\ipykernel_18692\3219207666.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  aggregated_interactions[column].fillna(pd.Timedelta(0), inplace=True)
C:\Users\jf79\AppData\Local\Temp\ipykernel_18692\3219207666.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

#########################################################################################################################################################################################
DataFrame: None


,direction,type,dateofqueue,groupname,queue,result,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time,isunder40s
0,Inbound,Call,2022-01-04,ACE and Admissions,ACE and Admissions,Answered,15,00.00:04:42,00.00:06:12,00.00:06:12,00.00:05:56,00.00:00:00,00.00:10:39,1.0
1,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Abandoned,2,00.00:00:00,00.00:02:00,00.00:02:00,00.00:00:00,00.00:00:00,00.00:00:00,1.0
2,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Answered,66,00.00:00:29,00.00:06:22,00.00:06:22,00.00:05:58,00.00:00:00,00.00:06:28,0.0
3,Inbound,Call,2022-01-04,CAS Team 1 6945,CAS Team 1 6945 Queue,Answered,3,00.00:01:00,00.00:01:40,00.00:01:40,00.00:01:10,00.00:00:00,00.00:02:10,0.0
4,Inbound,Call,2022-01-04,CAS Team 2 6946,CAS Team 2 6946 Queue,Answered,3,00.00:01:00,00.00:02:00,00.00:02:00,00.00:01:38,00.00:00:00,00.00:02:38,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191101,Outbound,Email,2025-02-19,Customer Services,Cost of Living Crisis Email,Sent,2,00.00:00:00,00.00:02:29,00.00:02:29,00.00:02:27,00.00:00:00,00.00:03:06,0.0
191102,Outbound,Email,2025-02-20,Customer Services,Cost of Living Crisis Email,Sent,1,00.00:00:00,00.00:01:13,00.00:01:13,00.00:01:12,00.00:00:00,00.00:02:13,0.0
191103,Outbound,Email,2025-02-21,Customer Services,Cost of Living Crisis Email,Sent,2,00.00:00:00,00.00:02:22,00.00:02:22,00.00:02:21,00.00:00:00,00.00:03:21,0.0
191104,Outbound,Email,2025-02-24,Customer Services,Cost of Living Crisis Email,Sent,1,00.00:00:00,00.00:01:00,00.00:01:00,00.00:01:00,00.00:00:00,00.00:02:00,0.0


Unique values per field:


,Field Name,No. of Unique Values
0,direction,2
1,type,3
2,dateofqueue,1142
3,groupname,47
4,queue,186
5,result,11
6,number_of_interactions,421
7,wrap_upduration,873
8,queue_time,19129
9,duration,19129



Number of duplicate rows:
0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191106 entries, 0 to 191105
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   direction               191106 non-null  object 
 1   type                    191106 non-null  object 
 2   dateofqueue             191106 non-null  object 
 3   groupname               191106 non-null  object 
 4   queue                   191106 non-null  object 
 5   result                  191106 non-null  object 
 6   number_of_interactions  191106 non-null  int64  
 7   wrap_upduration         191106 non-null  object 
 8   queue_time              191106 non-null  object 
 9   duration                191106 non-null  object 
 10  talk_time               191106 non-null  object 
 11  hold_time               191106 non-null  object 
 12  agent_handling_time     191106 non-null  object 
 13  isunder40s              191106 non-null  fl

None


Summary statistics:


,number_of_interactions,isunder40s
count,191106.000000,191106.000000
mean,12.310676,1.346546
std,33.705694,22.705985
min,1.000000,0.000000
25%,1.000000,0.000000
50%,3.000000,0.000000
75%,11.000000,1.000000
max,4154.000000,4154.000000


End of data validation
#########################################################################################################################################################################################



In [72]:
mf.export_to_csv(aggregated_interactions, df_name='Netcall Dashboard Data')

Exproting Netcall Dashboard Data to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry\Netcall Dashboard Data.csv

Successfully exported Netcall Dashboard Data to CSV
